# Import libraries

In [1]:
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, GroundWater, IrrigationManagement
from aquacrop.utils import prepare_weather, get_filepath
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pathlib
from pathlib import Path
import geopandas as gpd

import xarray as xr

import rasterio
from rasterio.plot import show

from matplotlib import pyplot

import numpy as np

import glob as glob

import rioxarray

from pathlib import Path

import zipfile
import rioxarray as rio 
from tqdm import tqdm
from datetime import datetime as dt
import datetime
import netCDF4 as nc
from netCDF4 import Dataset
import os, sys
import copy
import platform
import tempfile
import logging
from math import log10, cos, sin, asin, sqrt, exp, pi, radians
from bisect import bisect_left
import textwrap

import traceback

from datetime import date, timedelta

import json

from tqdm import tqdm

# Directories

In [2]:
HomeDir = '/home/c4ubuntu/c4gdata/SADC_Agri/'

In [3]:
DataDir = '/home/c4ubuntu/c4gdata/SADC_Agri/Data/'

# Functions

In [6]:
def get_plantdate_month(lat, lon, crop, irr_stat, weatherfile):
    calendar = xr.open_dataset(os.path.join(DataDir,'Crop/Crop_Calendar/{}_{}_ggcmi_crop_calendar_phase3_v1.01.nc4').format(crop, irr_stat))
    pnt_date = calendar.sel(lon=lon,lat=lat,  method="nearest")
    pnt_date = pnt_date.planting_day.values
    sdate = weatherfile.Date.dt.year.unique()[0]
    strt_date = date(int(sdate), 1, 1)
    plant_date = strt_date + timedelta(days=int(pnt_date) - 1)
    plant_date = plant_date.strftime("%m-%d")
    return(plant_date)

In [7]:
def get_plantcalendardate(lat, lon, crop, irr_stat, weatherfile):
    calendar = xr.open_dataset(os.path.join(DataDir,'Crop/Crop_Calendar/{}_{}_ggcmi_crop_calendar_phase3_v1.01.nc4').format(crop, irr_stat))
    pnt_date = calendar.sel(lon=lon,lat=lat,  method="nearest")
    pnt_date = pnt_date.planting_day.values
    sdate = weatherfile.Date.dt.year.unique()[0]
    strt_date = date(int(sdate), 1, 1)
    plant_date = strt_date + timedelta(days=int(pnt_date) - 1)
    plant_date = plant_date.strftime("%d-%m-%Y")
    return(plant_date)

In [8]:
def get_startdate(year):
    try:
        start_sim = str(year) + '-' + planting_dates.get(year)
        start_sim = pd.to_datetime(start_sim)
        start_sim = start_sim - pd.DateOffset(90)
        start_sim = start_sim.strftime('%Y/%m/%d')
    except:
        print('No planting date: ', year, planting_dates.get(year))
    return(start_sim)

In [9]:
def get_enddate(year):
    try:
        end_sim = str(year) + '-' + planting_dates.get(year)
        end_sim = pd.to_datetime(end_sim)
        end_sim = end_sim + pd.DateOffset(365)
        end_sim = end_sim.strftime('%Y/%m/%d')
    except:
        print('No planting date: ', year, planting_dates.get(year))
    return(end_sim)

In [10]:
##This function checks for the first date on which more than 25mm of rainfall is received in 7 days, followed by 40 mm in 14 days
def get_plantingdates(weatherfile, yearend):
    planting_dates = {}
    for year in weatherfile.Date.dt.year.unique():
        if year != yearend:
            start_window = f'{year}-08-01'
            end_window = f'{year+1}-04-01'
            year_clim = weatherfile.set_index('Date').loc[start_window:end_window]
            year_clim.reset_index(inplace=True)
            window_clim_01 = year_clim.set_index('Date').resample('7D').sum()
            window_clim_02 = year_clim.set_index('Date').resample('14D').sum()
            arex_01 = window_clim_01[start_window:end_window].loc[window_clim_01['Precipitation']>25]
            arex_02 = window_clim_02[start_window:end_window].loc[window_clim_02['Precipitation']>40]
            arex_03 = window_clim_01[start_window:end_window].loc[window_clim_01['Precipitation']>10]
            arex = [i for i in arex_01.index if i in arex_02.index]
            if len(arex) > 0:
                ar = arex[0]
            else:
                try:
                    ar = arex_01.index[0]
                except:
                    if len(arex_02) > 0:
                        ar = arex_02.index[0]
                    elif len(arex_03) > 0:
                        ar = arex_03.index[0]
            planting_dates[year] = ar.strftime("%Y/%m/%d")
    return(planting_dates)

In [11]:
def get_soil_profile(lon, lat, row, pen):
    #Soil Class
    pnt_class_name = row.get('SoilName')
    # Readily Evaporable Water
    readiliy_ew = {'SiltClay':12,'SandyClay':9,'ClayLoam':9,'SiltClayLoam':11,'SandyClayLoam':9,'Loam':10,'SandyLoam':9,'LoamySand':9,'Sand':6}
    point_rew = readiliy_ew.get(pnt_class_name)
    # Custom Soil
    point_soil = Soil(soil_type='custom', dz=[0.05]*2+[0.15]*2+[0.30]*2+[0.6]*2+[1.0]*2+[2.0]*2,adj_rew=1, rew=point_rew)
    
    depths = {'1':'0.5','2':'0.15','3':'0.30','4':'0.60','5':'1','6':'2'}
    
    for level in range(1,7):
        for var in ['SND','CLY']:
            if var == 'SND':
                sandtempdata = int(row.get('SND')[level])
               # sandtempdata = str(sandtempdata)
               # print(var, 'Level:',level, 'Value: ',sandtempdata)
            elif var == 'CLY':
                claytempdata = int(row.get('CLY')[level])
               # claytempdata = str(claytempdata)
               # print(var, 'Level:',level, 'Value: ',claytempdata)
        dep = depths.get('{}'.format(level))
        dep = float(dep)
        #point_soil.add_layer(thickness=dep,thWP=0.24,thFC=0.40,thS=0.50,Ksat=155,penetrability=50)
        point_soil.add_layer_from_texture(thickness=dep, Sand=sandtempdata, Clay=claytempdata, OrgMat=2.5, penetrability=pen)
        return(point_soil)

## Run Aquacrop

### Baseline

In [10]:
def run_aquacrop_model(data_type, 
                       model_name,
                       Scenario,
                       countries_points_dict, yearstart, yearend):
    
    Yields = pd.DataFrame()
    CropGrowth = pd.DataFrame()
    WaterStorage = pd.DataFrame()
    WaterFlux = pd.DataFrame()
    tracebacks = []
    
    for row in tqdm(countries_points_dict):
        lon = "{:.1f}".format(float(row.get('lon')))
        lat = "{:.1f}".format(float(row.get('lat')))

        # Country
        country = row.get('ADMIN')
        # Soil
        point_soil_name = row.get('SoilName')
        try:
            point_soil = get_soil_profile(lat=lat, lon=lon, row=row, pen=100)
            #print(point_soil.profile)
        except:
            traceback_output = traceback.format_exc()
            print('Soil profile didnt work', traceback_output)
            point_soil = Soil(soil_type=point_soil_name)

        # Root Zone Depth
        pnt_rootzoneD = row.get('RZD')

        # Climate
        clim_filename = os.path.join(DataDir,f'Climate/SiteFiles/{data_type}/{country}',
                                     f'{model_name}_{country}_{lat}_{lon}.csv')
        
        SiteFile = pd.read_csv(clim_filename,parse_dates=['Date'])
        SiteFile = SiteFile.loc[(SiteFile['Date']>=yearstart)&(SiteFile['Date']<=yearend)]
        SiteFile.drop(SiteFile.columns[[0]], axis=1, inplace=True)
        SiteFile.dropna(inplace=True)
        
        # Planting date
        try:
            planting_dates = get_plantingdates(weatherfile=SiteFile, yearend=yearend)
        except:
            planting_dates = []
            print('Could not get planting dates')

        # Run scenarios for a specific grid point
        for scenario in Scenario:
            # Scenario
            scene = str(scenario.get('Name'))
            # Cropname and Irrigation
            cropname = scenario.get('Crop')
            irr_status = scenario.get('Irrigation')

            # Initial water
            InitWC = InitialWaterContent(value=['FC'])

            # Irrigation
            irr_num = scenario.get('IrrNum')

            soilmt = scenario.get('SMT')       
            if soilmt is None:
                net_irrigation = IrrigationManagement(irrigation_method=int(irr_num))
            else:
                net_irrigation = IrrigationManagement(irrigation_method=int(irr_num),SMT=str(soilmt))

            # Run model for each year    
            for year in SiteFile.Date.dt.year.unique():
                if ((year > SiteFile.Date.dt.year.min())) and ((year < int(yearend))):  
                    # Planting date (the planting date may not be in the same year therefore we have to change the start and and time of the simulation)
                    try:
                        plant_date = planting_dates.get(year)
                    except:
                        plant_date = []
                        #print(f'Could not get plant date for {year}')

                    # Start and end Date
                    if len(plant_date) > 0:
                        # Get the planting month because that is how AquaCrop wants it
                        plant_month = pd.to_datetime(plant_date).strftime("%m/%d")
                        # Crop
                        # Crop
                        if cropname == 'Maize':
                            crop_sim = Crop('Maize', planting_date='{}'.format(plant_month),
                                         WP=scenario.get('WP'), Zmax=pnt_rootzoneD)
                        elif cropname == 'SugarCane':
                            crop_sim = Crop('{}'.format(cropname), planting_date='{}'.format(plant_month),
                                            WP=scenario.get('WP'),HI0=int(scenario.get('HI0')))
                        else:
                            crop_sim = Crop('{}'.format(cropname), planting_date='{}'.format(plant_month),
                                            WP=scenario.get('WP'))

                        #The start date is the same as the planting date 
                        start_date = pd.to_datetime(plant_date)
                        start_date = start_date.strftime("%Y/%m/%d")

                        #We make the end date a year later from the start date
                        end_date = pd.to_datetime(plant_date)+pd.DateOffset(370)
                        end_date = end_date.strftime("%Y/%m/%d")
                        PointFile = SiteFile.set_index('Date').loc[start_date:end_date].reset_index()

                        # Run Aquacrop
                        model = AquaCropModel(sim_start_time=start_date,
                                          sim_end_time=end_date,
                                          weather_df=SiteFile,
                                          soil=point_soil, 
                                          crop=crop_sim,
                                          initial_water_content=InitWC,
                                          irrigation_management=net_irrigation)
                        #model._initialize()
                        try:
                            model.run_model(till_termination=True)
                            out = model._outputs.final_stats
                            water_flux = model._outputs.water_flux
                            water_flux['year'] = year
                            water_flux['Lat'] = lat
                            water_flux['Lon'] = lon
                            water_flux['N_ID'] = lat+lon
                            water_store = model._outputs.water_storage
                            water_store['year'] = year
                            water_store['Lat'] = lat
                            water_store['Lon'] = lon
                            water_store['N_ID'] = lat+lon
                            crop_growth = model._outputs.crop_growth
                            crop_growth['year'] = year
                            crop_growth['Lat'] = lat
                            crop_growth['Lon'] = lon
                            crop_growth['N_ID'] = lat+lon
                            out['Lat'] = lat
                            out['Lon'] = lon
                            out['Scenario'] = scene
                            out['Country'] = country
                            out['N_ID'] = lat+lon
                            out['plant_date'] = '{}'.format(planting_dates.get(year))
                            if len(WaterFlux) == 0:
                                WaterFlux = water_flux
                            else:
                                WaterFlux = pd.concat([WaterFlux, water_flux], axis=0)
                            if len(WaterStorage) == 0:
                                WaterStorage = water_store
                            else:
                                WaterStorage = pd.concat([WaterStorage, water_store], axis=0)
                            if len(CropGrowth) == 0:
                                CropGrowth = crop_growth
                            else:
                                CropGrowth = pd.concat([CropGrowth, crop_growth], axis=0)
                            if len(Yields) == 0:
                                Yields = out
                            else:
                                Yields = pd.concat([Yields, out], axis=0)
                        except Exception as e:
                            traceback_output = traceback.format_exc()
                            tracebacks.append(f"Model run for {lat} {lon} in {year} failed with exception: {traceback_output}.{plant_date},{start_date},{end_date}")
                            #print(f"Model run for {lat} {lon} in {year} failed.",
                            #     plant_date, start_date, end_date)
                    else:
                        continue
                        #print('No planting')
    return Yields, CropGrowth, WaterStorage, WaterFlux, tracebacks

In [12]:
def run_aquacrop_model(data_type, 
                       model_name,
                       Scenario,
                       countries_points_dict, yearstart, yearend):
    
    Yields = pd.DataFrame()
    tracebacks = []
    
    for row in tqdm(countries_points_dict):
        lon = "{:.1f}".format(float(row.get('lon')))
        lat = "{:.1f}".format(float(row.get('lat')))

        # Country
        country = row.get('ADMIN')
        # Soil
        point_soil_name = row.get('SoilName')
        try:
            point_soil = get_soil_profile(lat=lat, lon=lon, row=row, pen=100)
            #print(point_soil.profile)
        except:
            traceback_output = traceback.format_exc()
            print('Soil profile didnt work', traceback_output)
            point_soil = Soil(soil_type=point_soil_name)

        # Root Zone Depth
        pnt_rootzoneD = row.get('RZD')

        # Climate
        clim_filename = os.path.join(DataDir,f'Climate/SiteFiles/{data_type}/{country}',
                                     f'{model_name}_{country}_{lat}_{lon}.csv')
        
        SiteFile = pd.read_csv(clim_filename,parse_dates=['Date'])
        SiteFile = SiteFile.loc[(SiteFile['Date']>=yearstart)&(SiteFile['Date']<=yearend)]
        SiteFile.drop(SiteFile.columns[[0]], axis=1, inplace=True)
        SiteFile.dropna(inplace=True)
        
        # Planting date
        try:
            planting_dates = get_plantingdates(weatherfile=SiteFile, yearend=yearend)
        except:
            planting_dates = []
            print('Could not get planting dates')

        # Run scenarios for a specific grid point
        for scenario in Scenario:
            # Scenario
            scene = str(scenario.get('Name'))
            # Cropname and Irrigation
            cropname = scenario.get('Crop')
            irr_status = scenario.get('Irrigation')

            # Initial water
            InitWC = InitialWaterContent(value=['FC'])

            # Irrigation
            irr_num = scenario.get('IrrNum')

            soilmt = scenario.get('SMT')       
            if soilmt is None:
                net_irrigation = IrrigationManagement(irrigation_method=int(irr_num))
            else:
                net_irrigation = IrrigationManagement(irrigation_method=int(irr_num),SMT=str(soilmt))

            # Run model for each year    
            for year in SiteFile.Date.dt.year.unique():
                if ((year > SiteFile.Date.dt.year.min())) and ((year < int(yearend))):  
                    # Planting date (the planting date may not be in the same year therefore we have to change the start and and time of the simulation)
                    try:
                        plant_date = planting_dates.get(year)
                    except:
                        plant_date = []
                        #print(f'Could not get plant date for {year}')

                    # Start and end Date
                    if len(plant_date) > 0:
                        # Get the planting month because that is how AquaCrop wants it
                        plant_month = pd.to_datetime(plant_date).strftime("%m/%d")
                       
                        # Crop
                        if cropname == 'Maize':
                            crop_sim = Crop('Maize', planting_date='{}'.format(plant_month),
                                         WP=scenario.get('WP'), Zmax=pnt_rootzoneD)
                        elif cropname == 'SugarCane':
                            crop_sim = Crop('{}'.format(cropname), planting_date='{}'.format(plant_month),
                                            WP=scenario.get('WP'),HI0=int(scenario.get('HI0')))
                        else:
                            crop_sim = Crop('{}'.format(cropname), planting_date='{}'.format(plant_month),
                                            WP=scenario.get('WP'))

                        #The start date is the same as the planting date 
                        start_date = pd.to_datetime(plant_date)
                        start_date = start_date.strftime("%Y/%m/%d")

                        #We make the end date a year later from the start date
                        end_date = pd.to_datetime(plant_date)+pd.DateOffset(370)
                        end_date = end_date.strftime("%Y/%m/%d")
                        PointFile = SiteFile.set_index('Date').loc[start_date:end_date].reset_index()

                        # Run Aquacrop
                        model = AquaCropModel(sim_start_time=start_date,
                                          sim_end_time=end_date,
                                          weather_df=SiteFile,
                                          soil=point_soil, 
                                          crop=crop_sim,
                                          initial_water_content=InitWC,
                                          irrigation_management=net_irrigation)
                        #model._initialize()
                        try:
                            model.run_model(till_termination=True)
                            out = model._outputs.final_stats
                            out['Lat'] = lat
                            out['Lon'] = lon
                            out['Scenario'] = scene
                            out['Country'] = country
                            out['N_ID'] = lat+lon
                            out['plant_date'] = '{}'.format(planting_dates.get(year))
                          
                            if len(Yields) == 0:
                                Yields = out
                            else:
                                Yields = pd.concat([Yields, out], axis=0)
                        except Exception as e:
                            traceback_output = traceback.format_exc()
                            tracebacks.append(f"Model run for {lat} {lon} in {year} failed with exception: {traceback_output}.{plant_date},{start_date},{end_date}")
                    else:
                        continue
                        #print('No planting')
    return Yields, tracebacks

In [13]:
country = 'luangwa'
with open(os.path.join(DataDir,'GIS','Gridpoints',f'{country}_points_soils_dict.json')) as json_file:
    countries_points_dict = json.load(json_file)

In [14]:
len(countries_points_dict)

1098

In [15]:
experiment = 'Baseline'

experiment_prefix = 'AgERA5'

data_type = 'AgERA5'

country = 'Zambia'

models = ['AgERA5']

Scenario =  [{'Name':'Low_Input','IrrNum':'0','Crop':'Sorghum', 'Irrigation':'rf', 'WP':23},
             {'Name':'High_Input','IrrNum':'4','SMT':'60','Crop':'Sorghum', 'Irrigation':'ir','WP':23},
            {'Name':'Low_Input','IrrNum':'0','Crop':'Maize', 'Irrigation':'rf', 'WP':23}, 
            {'Name':'High_Input','IrrNum':'4','SMT':'60','Crop':'Maize', 'Irrigation':'ir','WP':23},
            {'Name':'Low_Input','IrrNum':'0','Crop':'SugarCane', 'Irrigation':'rf','WP':40, 'HI0':2000},
            {'Name':'High_Input','IrrNum':'4','SMT':'60','Crop':'SugarCane', 'Irrigation':'ir','WP':40, 'HI0':2000}]

yearstart = '1979'
yearend = '2020'

for model_name in models:
    print(model_name)
    Yields, tracebacks = run_aquacrop_model(data_type, 
                                            model_name, 
                                            Scenario, 
                                            countries_points_dict,
                                            yearstart, yearend)
    Yields.to_csv(os.path.join(DataDir,f'Output/CSVs/{experiment}/{data_type}','{}_{}_Yields_{}_{}.csv'.format(country,experiment_prefix,model_name,str(dt.now()))))

AgERA5


100%|█████████████████████████████████████| 1098/1098 [5:37:52<00:00, 18.46s/it]
